<a href="https://colab.research.google.com/github/abhinav3/cross_batch_memory/blob/main/PML_Issue_421.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pytorch-metric-learning

     |████████████████████████████████| 106 kB 5.6 MB/s 


In [4]:
import torch
import torchvision
from pytorch_metric_learning import losses, distances, regularizers
import tqdm
import psutil
import sys
import os

# https://discuss.pytorch.org/t/inspect-memory-usage/7046
def cpuStats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
    return memoryUse

device = torch.device("cuda")
model = torchvision.models.resnet18().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
loss_fn = losses.TripletMarginLoss(distance = distances.CosineSimilarity(), margin=0.2, embedding_regularizer = regularizers.LpRegularizer())
dataset = torchvision.datasets.FakeData(transform=torchvision.transforms.ToTensor())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=2)

##########
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
##########
for epoch in range(10):
    train_losses = AverageMeter()
    print("epoch", epoch)
    pbar = tqdm.tqdm(dataloader)
    for imgs, labels in pbar:
        pbar.set_description(f"CPU memory usage = {cpuStats()} GB")
        embeddings = model(imgs.to(device))
        loss = loss_fn(embeddings, labels)
        train_losses.update(loss, imgs.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

epoch 0


CPU memory usage = 3.0142173767089844 GB: 100%|██████████| 32/32 [00:07<00:00,  4.49it/s]


epoch 1


CPU memory usage = 3.01068115234375 GB: 100%|██████████| 32/32 [00:06<00:00,  4.61it/s]


epoch 2


...

CPU memory usage = 3.0136375427246094 GB: 100%|██████████| 32/32 [00:07<00:00,  4.57it/s]


epoch 8


CPU memory usage = 3.0129661560058594 GB: 100%|██████████| 32/32 [00:06<00:00,  4.57it/s]


epoch 9


CPU memory usage = 3.0127830505371094 GB: 100%|██████████| 32/32 [00:06<00:00,  4.58it/s]
